In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
countVectorizer = pickle.load(open('BinaryCountVectorizer.pkl', 'rb'))
tfidfVectorizer = pickle.load(open('TfidfVectorizer.pkl', 'rb'))

tfidfModel = pickle.load(open('TFIDFVotingClassifier.pkl', 'rb'))
featuresModel = pickle.load(open('CustomFeaturesVotingClassifier.pkl', 'rb'))

In [3]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

from fuzzywuzzy import fuzz
from scipy.sparse import hstack
from scipy.spatial.distance import hamming, cosine

STOP_WORDS = stopwords.words("english")
ps = PorterStemmer()

def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    x = re.sub(r'\W', ' ', str(x))
    
    x = re.sub(r"\s+", ' ', str(x))
    
    x = ' '.join(ps.stem(word) for word in x.split())
    
    soup = BeautifulSoup(str(x))
    x = soup.get_text()

    x = ' '.join([word for word in x.split() if word not in STOP_WORDS])
    
    return x

def lcs(X , Y): 
    m = len(X) 
    n = len(Y) 

    L = [[None]*(n+1) for i in range(m+1)] 
  
    for i in range(m+1): 
        for j in range(n+1): 
            if i == 0 or j == 0 : 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1]+1
            else: 
                L[i][j] = max(L[i-1][j] , L[i][j-1]) 
  
    return L[m][n] 

D:\Users\Ritvik\Anaconda3\envs\datascience\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
def generateFeatures(text1, text2):
 
    abs_token_diff = abs(len(text1.split())-len(text2.split()))
    avg_num_token = (len(text1.split())+len(text2.split()))/2
    rel_token_diff = abs_token_diff/avg_num_token
    token_intersection = len(set(text1.split())&set(text2.split()))
    token_union = len(set(text1.split())|set(text2.split()))
    jaccard_similarity_token = token_intersection/token_union
    lcs_token = lcs(text1.split(), text2.split())
    lcs_token_ratio = lcs_token/avg_num_token
    
    text1_clean = preprocess(text1)
    text2_clean = preprocess(text2)
    
    abs_word_diff = abs(len(text1_clean.split())-len(text2_clean.split()))
    avg_num_word = (len(text1_clean.split())+len(text2_clean.split()))/2
    rel_word_diff = abs_word_diff/avg_num_word
    word_intersection = len(set(text1_clean.split())&set(text2_clean.split()))
    word_union = len(set(text1_clean.split())|set(text2_clean.split()))
    jaccard_similarity_word = word_intersection/word_union
    lcs_word = lcs(text1_clean.split(), text2_clean.split())
    lcs_word_ratio = lcs_word/avg_num_word
    
    fuzz_simple_ratio = fuzz.ratio(str(text1_clean).split(), str(text2_clean).split())
    fuzz_partial_ratio = fuzz.partial_ratio(str(text1_clean).split(), str(text2_clean).split())
    fuzz_token_sort_ratio = fuzz.token_sort_ratio(str(text1_clean).split(), str(text2_clean).split())
    fuzz_token_set_ratio = fuzz.token_set_ratio(str(text1_clean).split(), str(text2_clean).split())
    
    v1 = countVectorizer.transform([text1_clean])
    v2 = countVectorizer.transform([text2_clean])
    
    hamming_distance = hamming(v1.toarray()[0], v2.toarray()[0])
    
    v1 = tfidfVectorizer.transform([text1_clean])
    v2 = tfidfVectorizer.transform([text2_clean])
    
    cosine_distance = cosine(v1.toarray()[0], v2.toarray()[0])
    
    features = {
        'abs_token_diff': abs_token_diff, 'avg_num_token': avg_num_token, 'rel_token_diff': rel_token_diff,
        'token_intersection': token_intersection, 'token_union': token_union, 
        'jaccard_similarity_token': jaccard_similarity_token, 'lcs_token': lcs_token, 'lcs_token_ratio': lcs_token_ratio,
        'abs_word_diff': abs_word_diff, 'avg_num_word': avg_num_word, 'rel_word_diff': rel_word_diff,
        'word_intersection': word_intersection, 'word_union': word_union, 
        'jaccard_similarity_word': jaccard_similarity_word, 'lcs_word': lcs_word, 'lcs_word_ratio': lcs_word_ratio,
        'fuzz_simple_ratio': fuzz_simple_ratio, 'fuzz_partial_ratio': fuzz_partial_ratio, 
        'fuzz_token_sort_ratio': fuzz_token_sort_ratio, 'fuzz_token_set_ratio': fuzz_token_set_ratio,
        'hamming_distance': hamming_distance, 'cosine_distance': cosine_distance
    }
    
    features = pd.DataFrame(features, index=[0])
    
    vects = hstack([v1, v2])
    
    return features, vects

In [5]:
def getPredictions(feats, vects):
    labels = {
        0: 'unique', 1: 'duplicate'
    }
    
    probabilities = pd.DataFrame(np.concatenate([
        tfidfModel.estimators_[0].predict_proba(vects),
        tfidfModel.estimators_[1].predict_proba(vects),
        tfidfModel.estimators_[2].predict_proba(vects),
        tfidfModel.estimators_[3].predict_proba(vects),
        featuresModel.estimators_[0].predict_proba(feats),
        featuresModel.estimators_[1].predict_proba(feats),
        featuresModel.estimators_[2].predict_proba(feats)
    ], axis=0), index=['LR vects 1', 'LR vects 10', 'LR vects 100', 'SGD vects', 'LR', 'XGB', 'GB'], 
                                 columns=['unique', 'duplicate'])
    
    results = probabilities['duplicate'].apply(lambda x : labels[round(x)])
    
    return probabilities, results
    

In [6]:
q1 = 'hello how are you?'
q2 = 'who the hell are you?'

In [7]:
feats, vects = generateFeatures(q1, q2)
p, r = getPredictions(feats, vects)

In [8]:
p

,unique,duplicate
LR vects 1,0.813215,0.186785
LR vects 10,0.751844,0.248156
LR vects 100,0.662149,0.337851
SGD vects,0.654381,0.345619
LR,0.164120,0.835880
XGB,0.725316,0.274684
GB,0.742827,0.257173


In [9]:
r

LR vects 1         unique
LR vects 10        unique
LR vects 100       unique
SGD vects          unique
LR              duplicate
XGB                unique
GB                 unique
Name: duplicate, dtype: object